# Detector simulation
In this notebook we load a track dataset generated by `edep-sim` and we calculate the ADC counts corresponding to each pixel. The result is exported to a HDF5 file.

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
# This is need so you can import larndsim without doing python setup.py install
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [ ]:
from math import ceil
from time import time

import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mpl_toolkits.mplot3d.art3d as art3d

import numpy as np
import cupy as cp
import eagerpy as ep
#import fire
import h5py

from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100


from tqdm.notebook import tqdm

In [ ]:
from larndsim import consts
import importlib
importlib.reload(consts)
consts.load_detector_properties("../larndsim/detector_properties/module0.yaml",
                                "../larndsim/pixel_layouts/multi_tile_layout-2.2.16.yaml")

from larndsim import quenching, drifting, detsim, pixels_from_track, fee

from numpy.lib import recfunctions as rfn
import torch

def torch_from_structured(tracks):
    tracks_np = rfn.structured_to_unstructured(tracks, copy=True, dtype=np.float32)
    return torch.from_numpy(tracks_np).float().cuda()

def structered_from_torch(tracks_torch, dtype):
    return rfn.unstructured_to_structured(tracks_torch.cpu().numpy(), dtype=dtype)

In [ ]:
from larndsim.sim_with_grad import sim_with_grad
sim = sim_with_grad(track_chunk=64, pixel_chunk=4)
sim.load_detector_properties("../larndsim/detector_properties/module0.yaml",
                             "../larndsim/pixel_layouts/multi_tile_layout-2.2.16.yaml")

### Dataset import
First of all we load the `edep-sim` output. For this sample we need to invert $z$ and $y$ axes.

In [ ]:
unique_pix = cp.empty((0,2))
pixels_signals = cp.zeros((len(unique_pix), len(consts.time_ticks)*3))

In [ ]:
with h5py.File('module0_corsika.h5', 'r') as f:
    tracks = np.array(f['segments'])

x_start = np.copy(tracks['x_start'] )
x_end = np.copy(tracks['x_end'])
x = np.copy(tracks['x'])

tracks['x_start'] = np.copy(tracks['z_start'])
tracks['x_end'] = np.copy(tracks['z_end'])
tracks['x'] = np.copy(tracks['z'])

tracks['z_start'] = x_start
tracks['z_end'] = x_end
tracks['z'] = x

selected_tracks = tracks[30:40]

### Quenching and drifting
We calculate the number of electrons after recombination (`quenching` module) and the position and number of electrons after drifting (`drifting` module).

In [ ]:
TPB = 256
BPG = ceil(selected_tracks.shape[0] / TPB)
import importlib
importlib.reload(drifting)

selected_tracks_torch = torch_from_structured(np.copy(selected_tracks))

quenching.quench[BPG,TPB](selected_tracks, consts.birks)
drifting.drift[BPG,TPB](selected_tracks)

In [ ]:
selected_tracks_quench = sim.quench(selected_tracks_torch, consts.birks, fields=selected_tracks.dtype.names)

#Truncation makes gradients vanish - do here for comparisons
n_elec_idx = selected_tracks.dtype.names.index('n_electrons')
selected_tracks_quench[:, n_elec_idx] = selected_tracks_quench[:, n_elec_idx].trunc()

selected_tracks_drift = sim.drift(selected_tracks_quench, fields=selected_tracks.dtype.names)
selected_tracks_drift[:, n_elec_idx] = selected_tracks_drift[:, n_elec_idx].trunc()

In [ ]:
(selected_tracks_drift[:, selected_tracks.dtype.names.index('n_electrons')].cpu().numpy()
  == selected_tracks['n_electrons']).all()

We find the pixels intersected by the projection of the tracks on the anode plane using the Bresenham's algorithm. We also take into account the neighboring pixels, due to the transverse diffusion of the charges.

In [ ]:
importlib.reload(pixels_from_track)

# Here we build a map between tracks and event IDs
unique_eventIDs = cp.unique(selected_tracks['eventID'])
event_id_map = cp.searchsorted(unique_eventIDs,cp.asarray(selected_tracks['eventID']))
event_id_map_torch = torch.from_numpy(event_id_map.get()).cuda()

longest_pix = ceil(max(selected_tracks["dx"])/consts.pixel_pitch)
max_radius = ceil(max(selected_tracks["tran_diff"])*5/consts.pixel_pitch)

MAX_PIXELS = int((longest_pix*4+6)*max_radius*1.5)
MAX_ACTIVE_PIXELS = int(longest_pix*1.5)

active_pixels = cp.full((selected_tracks.shape[0], MAX_ACTIVE_PIXELS, 2), -1, dtype=np.int32)
neighboring_pixels = cp.full((selected_tracks.shape[0], MAX_PIXELS, 2), -1, dtype=np.int32)
n_pixels_list = cp.zeros(shape=(selected_tracks.shape[0]))
TPB = 128
BPG = ceil(selected_tracks.shape[0] / TPB)
pixels_from_track.get_pixels[BPG,TPB](selected_tracks,
                                      active_pixels,
                                      neighboring_pixels,
                                      n_pixels_list,
                                      max_radius+1)

In [ ]:
active_pixels_torch, neighboring_pixels_torch, n_pixels_list_ep = sim.get_pixels(selected_tracks_drift,
                                                                                fields=selected_tracks.dtype.names)
print(neighboring_pixels_torch.shape)
print(neighboring_pixels.shape)

## Bug?
Neighboring pixels is different for one of the track segments. Running _just_ that track segment vs running a set of track segments gives two different results in the cupy setup. The single track segment cupy result matches the eagerpy setup.

In [ ]:
np.where((neighboring_pixels_torch.cpu().numpy() != neighboring_pixels.get()))

In [ ]:
def cupy_unique_axis0(array):
    # axis is still not supported for cupy.unique, this
    # is a workaround
    if len(array.shape) != 2:
        raise ValueError("Input array must be 2D.")
    sortarr     = array[cp.lexsort(array.T[::-1])]
    mask        = cp.empty(array.shape[0], dtype=cp.bool_)
    mask[0]     = True
    mask[1:]    = cp.any(sortarr[1:] != sortarr[:-1], axis=1)
    return sortarr[mask]

shapes = neighboring_pixels.shape
joined = neighboring_pixels.reshape(shapes[0]*shapes[1],2)
this_unique_pix = cupy_unique_axis0(joined)
this_unique_pix = this_unique_pix[(this_unique_pix[:,0] != -1) & (this_unique_pix[:,1] != -1),:]

In [ ]:
unique_pix = cp.concatenate((unique_pix, this_unique_pix),axis=0)

### Charge distribution calculation
Here we calculate the current induced by each track on the pixels, taking into account longitudinal and transverse diffusion. The track segment is parametrized as:
\begin{align}
x'(r') &=x_s + \frac{\Delta x}{\Delta r}r'\\
y'(r') &=y_s + \frac{\Delta y}{\Delta r}r'\\
z'(r') &=z_s + \frac{\Delta z}{\Delta r}r',
\end{align}
where $\Delta r$ is the segment length. Here we assume $z$ as the drift direction.
The diffused charge distribution is calculated with the following integral:
\begin{equation}
\rho(x,y,z) = \frac{Q}{\sqrt{(2\pi)^3}\sigma_x\sigma_y\sigma_z\Delta r}\exp\left[-\frac{(x-x_s)^2}{2\sigma_x^2}-\frac{(y-y_s)^2}{2\sigma_y^2}-\frac{(z-z_s)^2}{2\sigma_z^2}\right]\int^{r'=\Delta r}_{r'=0}dr'\exp[-(ar'^2+br')],
\end{equation}
where 
\begin{align}
a &= \left[\left(\frac{\Delta x}{\Delta r}\right)^2\frac{1}{2\sigma_x^2} + \left(\frac{\Delta y}{\Delta r}\right)^2\frac{1}{2\sigma_y^2} + \left(\frac{\Delta z}{\Delta r}\right)^2\frac{1}{2\sigma_z^2} \right]\\
b &= -\left[\frac{(x-x_s)}{\sigma_x^2}\frac{\Delta x}{\Delta r}+
\frac{(y-y_s)}{\sigma_y^2}\frac{\Delta y}{\Delta r}+
\frac{(z-z_s)}{\sigma_z^2}\frac{\Delta z}{\Delta r}\right].
\end{align}

The simmetry of the transverse diffusion along the track allows to take a slice on the $xy$ plane and solve the integral once at a fixed $z$ coordinate (e.g. at $z_{m} = (z_s+z_e)/2$) and re-use it at other $z$ coordinates away from the endpoints (where $\rho(x,y,z)$ varies along $z$ so must be calculated at each $z$). 

In [ ]:
importlib.reload(detsim)

max_length = cp.array([0])
track_starts = cp.empty(selected_tracks.shape[0])
threadsperblock = 128
blockspergrid = ceil(selected_tracks.shape[0] / threadsperblock)
detsim.time_intervals[blockspergrid,threadsperblock](track_starts, max_length,  event_id_map, selected_tracks)

In [ ]:
track_starts_torch, max_length_torch = sim.time_intervals(event_id_map_torch, 
                                                          selected_tracks_drift, 
                                                          fields=selected_tracks.dtype.names)

print(track_starts_torch, max_length_torch)
print(track_starts, max_length)

In [ ]:
signals = cp.zeros((selected_tracks.shape[0],
                    neighboring_pixels.shape[1],
                    cp.asnumpy(max_length)[0]), dtype=np.float32)
threadsperblock = (1,1,64)

importlib.reload(detsim)

blockspergrid_x = ceil(signals.shape[0] / threadsperblock[0])
blockspergrid_y = ceil(signals.shape[1] / threadsperblock[1])
blockspergrid_z = ceil(signals.shape[2] / threadsperblock[2])
blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)
detsim.tracks_current[blockspergrid,threadsperblock](signals,
                                                     neighboring_pixels,
                                                     selected_tracks)
print(signals.shape)

In [ ]:
signals_ep = sim.tracks_current(neighboring_pixels_torch, selected_tracks_drift, 
                                max_length_torch,
                                fields=selected_tracks.dtype.names)
print(signals_ep.shape)

In [ ]:
pixel_index_map = cp.full((selected_tracks.shape[0], neighboring_pixels.shape[1]), -1)
try:
    compare = neighboring_pixels[..., np.newaxis, :] == unique_pix
    indices = cp.where(cp.logical_and(compare[..., 0], compare[..., 1]))
except cp.cuda.memory.OutOfMemoryError:
    print("out of memory")
pixel_index_map[indices[0], indices[1]] = indices[2]

In [ ]:
this_pixels_signals = cp.zeros((len(this_unique_pix), len(consts.time_ticks)*3))
pixels_signals = cp.concatenate((pixels_signals, this_pixels_signals), axis=0)

In [ ]:
blockspergrid_x = ceil(signals.shape[0] / threadsperblock[0])
blockspergrid_y = ceil(signals.shape[1] / threadsperblock[1])
blockspergrid_z = ceil(signals.shape[2] / threadsperblock[2])
blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)
this_pixels_signals = cp.zeros((len(this_unique_pix), len(consts.time_ticks)*3))
pixels_signals = cp.concatenate((pixels_signals, this_pixels_signals), axis=0)
detsim.sum_pixel_signals[blockspergrid,threadsperblock](pixels_signals,
                                                        signals,
                                                        track_starts,
                                                        pixel_index_map)

In [ ]:
currents = pixels_signals.sum(axis=1)*sim.t_sampling/sim.e_charge

In [ ]:
sum(currents)/sum(selected_tracks['n_electrons'])

In [ ]:
unique_pix_torch = torch.empty((0, 2))
pixels_signals_torch = torch.zeros((len(unique_pix_torch), len(sim.time_ticks)*3))

shapes_torch = neighboring_pixels_torch.shape
joined_torch = neighboring_pixels_torch.reshape(shapes_torch[0]*shapes_torch[1], 2)

this_unique_pix_torch = torch.unique(joined_torch, dim=0)
this_unique_pix_torch = this_unique_pix_torch[(this_unique_pix_torch[:,0] != -1) & (this_unique_pix_torch[:,1] != -1),:]
unique_pix_torch = torch.cat((unique_pix_torch, this_unique_pix_torch),dim=0)

this_pixels_signals_torch = torch.zeros((len(this_unique_pix_torch), len(sim.time_ticks)*3))
pixels_signals_torch = torch.cat((pixels_signals_torch, this_pixels_signals_torch), dim=0)

pixel_index_map_torch = torch.full((selected_tracks.shape[0], neighboring_pixels_torch.shape[1]), -1)
compare_torch = (neighboring_pixels_torch[..., np.newaxis, :] == unique_pix_torch)

indices_torch = torch.where(torch.logical_and(compare_torch[..., 0], compare_torch[...,1]))
pixel_index_map_torch[indices_torch[0], indices_torch[1]] = indices_torch[2]

In [ ]:
pixels_signals_torch = sim.sum_pixel_signals(pixels_signals_torch,
                                             signals_ep,
                                            track_starts_torch,
                                            pixel_index_map_torch)

currents_torch = pixels_signals_torch.sum(dim=1)*sim.t_sampling/sim.e_charge

In [ ]:
sum(currents_torch)/sum(selected_tracks['n_electrons'])

In [ ]:
track = selected_tracks[0]
pixel_plane = track['pixel_plane']
z_anode = consts.tpc_borders[pixel_plane][2][0]
z = (track['z_start']+track['z_end'])/2
drift_distance = abs(z - z_anode)
drift_start = abs(min(track["z_start"],track["z_end"]) - z_anode)
drift_end = abs(max(track["z_start"],track["z_end"]) - z_anode) 
drift_time = drift_distance / consts.vdrift
drift_start_time = drift_start / consts.vdrift
drift_end_time = drift_end / consts.vdrift

### 3D event display

In [ ]:
%matplotlib notebook

cmap = cm.Spectral_r
norm = colors.Normalize(vmin=0, vmax=256)
m = cm.ScalarMappable(norm=norm, cmap=cmap)

cmap = cm.viridis
norm_curr = colors.LogNorm(vmin=min(currents[currents>0]), vmax=max(currents))
m_curr = cm.ScalarMappable(norm=norm_curr, cmap=cmap)
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

for it,t in enumerate(selected_tracks):
    if it == 0:
        ax.plot((t["x_start"], t["x_end"]), 
            (t["z_start"], t["z_end"]),
            (t["y_start"], t["y_end"]),
            c='r',
            lw=1,
            alpha=1,
            zorder=10,
            label='Geant4 detector segment')
    else:
        ax.plot((t["x_start"], t["x_end"]), 
                (t["z_start"], t["z_end"]),
                (t["y_start"], t["y_end"]),
                c='r',
                lw=1,
                alpha=1,
                zorder=9999)
        
    ax.plot((t["x_start"], t["x_end"]), 
            (consts.tpc_borders[t['pixel_plane']][2][0], consts.tpc_borders[t['pixel_plane']][2][0]),
            (t["y_start"], t["y_end"]),
            c='r',
            lw=1,
            ls=':',
            alpha=1,
            zorder=9999)
        
for ip, p in enumerate(unique_pix):
    x_rect, y_rect = detsim.get_pixel_coordinates(p.get())
    pixel_plane = int(p[0] // consts.n_pixels[0])
    row = int(pixel_plane // 4)
    column = int(pixel_plane % 4)
    if currents[ip] > 0:
        rect = plt.Rectangle((x_rect, y_rect),
                             consts.pixel_pitch, consts.pixel_pitch,
                             linewidth=0.1, fc=m_curr.to_rgba(currents[ip].get()),
                             edgecolor='gray')
        ax.add_patch(rect)
        art3d.pathpatch_2d_to_3d(rect, z=consts.tpc_borders[pixel_plane][2][0], zdir="y")
        
anode1 = plt.Rectangle((consts.tpc_borders[0][0][0], consts.tpc_borders[0][1][0]),
                        consts.tpc_borders[0][0][1]-consts.tpc_borders[0][0][0], 
                        consts.tpc_borders[0][1][1]-consts.tpc_borders[0][1][0],
                       linewidth=1, fc='none',
                       edgecolor='gray', label=('Pixel' if ip == 5 else ''))
ax.add_patch(anode1)
art3d.pathpatch_2d_to_3d(anode1, z=consts.tpc_borders[0][2][0], zdir="y")

anode2 = plt.Rectangle((consts.tpc_borders[0][0][0], consts.tpc_borders[0][1][0]),
                        consts.tpc_borders[0][0][1]-consts.tpc_borders[0][0][0], 
                        consts.tpc_borders[0][1][1]-consts.tpc_borders[0][1][0],
                       linewidth=1, fc='none',
                       edgecolor='gray', label=('Pixel' if ip == 5 else ''))
ax.add_patch(anode2)
art3d.pathpatch_2d_to_3d(anode2, z=consts.tpc_borders[1][2][0], zdir="y")

cathode = plt.Rectangle((consts.tpc_borders[0][0][0], consts.tpc_borders[0][1][0]),
                        consts.tpc_borders[0][0][1]-consts.tpc_borders[0][0][0], 
                        consts.tpc_borders[0][1][1]-consts.tpc_borders[0][1][0],
                        linewidth=1, fc='gray', alpha=0.5,
                        edgecolor='gray', label=('Pixel' if ip == 5 else ''))
ax.add_patch(cathode)
art3d.pathpatch_2d_to_3d(cathode, z=0, zdir="y")

ax.plot((consts.tpc_borders[0][0][0],consts.tpc_borders[0][0][0]),(consts.tpc_borders[0][2][0],consts.tpc_borders[1][2][0]),
        (consts.tpc_borders[0][1][0],consts.tpc_borders[0][1][0]), lw=1,color='gray')

ax.plot((consts.tpc_borders[0][0][0],consts.tpc_borders[0][0][0]),(consts.tpc_borders[0][2][0],consts.tpc_borders[1][2][0]),
        (consts.tpc_borders[0][1][1],consts.tpc_borders[0][1][1]), lw=1,color='gray')

ax.plot((consts.tpc_borders[0][0][1],consts.tpc_borders[0][0][1]),(consts.tpc_borders[0][2][0],consts.tpc_borders[1][2][0]),
        (consts.tpc_borders[0][1][0],consts.tpc_borders[0][1][0]), lw=1,color='gray')

ax.plot((consts.tpc_borders[0][0][1],consts.tpc_borders[0][0][1]),(consts.tpc_borders[0][2][0],consts.tpc_borders[1][2][0]),
        (consts.tpc_borders[0][1][1],consts.tpc_borders[0][1][1]), lw=1,color='gray')

# ax.set_ylim(consts.module_borders[pixel_plane][2][0],50)
ax.set_xlim(consts.tpc_borders[0][0][0],consts.tpc_borders[1][0][1])
ax.set_ylim(consts.tpc_borders[0][2][0],consts.tpc_borders[1][2][0])
ax.set_zlim(consts.tpc_borders[0][1][0],consts.tpc_borders[1][1][1])

ax.set_box_aspect((2,2,4))
ax.grid(False)
ax.xaxis.set_major_locator(plt.MaxNLocator(3))
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
# ax.set_axis_off()
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

ax.view_init(azim=20)
# def rotate(angle):
#     ax.view_init(azim=angle)
    
# from matplotlib import animation

# rot_animation = animation.FuncAnimation(fig, rotate, frames=np.arange(0,362,2),interval=50)

ax.set_ylabel("z [cm]")
ax.set_xlabel("x [cm]")
ax.set_zlabel("y [cm]")
_ = plt.colorbar(m_curr,fraction=0.035, pad=0.05,label='Induced current integral [# electrons]')


In [ ]:
%matplotlib notebook

cmap = cm.Spectral_r
norm = colors.Normalize(vmin=0, vmax=256)
m = cm.ScalarMappable(norm=norm, cmap=cmap)

cmap = cm.viridis
norm_curr = colors.LogNorm(vmin=min(currents_torch[currents_torch>0]), vmax=max(currents_torch))
m_curr = cm.ScalarMappable(norm=norm_curr, cmap=cmap)
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

for it,t in enumerate(selected_tracks):
    if it == 0:
        ax.plot((t["x_start"], t["x_end"]), 
            (t["z_start"], t["z_end"]),
            (t["y_start"], t["y_end"]),
            c='r',
            lw=1,
            alpha=1,
            zorder=10,
            label='Geant4 detector segment')
    else:
        ax.plot((t["x_start"], t["x_end"]), 
                (t["z_start"], t["z_end"]),
                (t["y_start"], t["y_end"]),
                c='r',
                lw=1,
                alpha=1,
                zorder=9999)
        
    ax.plot((t["x_start"], t["x_end"]), 
            (sim.tpc_borders[t['pixel_plane']][2][0], sim.tpc_borders[t['pixel_plane']][2][0]),
            (t["y_start"], t["y_end"]),
            c='r',
            lw=1,
            ls=':',
            alpha=1,
            zorder=9999)

x_r_all, y_r_all = sim.get_pixel_coordinates(ep.astensor(unique_pix_torch))
for ip, p in enumerate(unique_pix_torch):
    x_rect = x_r_all[ip, 0].numpy()
    y_rect = y_r_all[ip, 0].numpy()
    pixel_plane = int(p[0] // sim.n_pixels[0])
    row = int(pixel_plane // 4)
    column = int(pixel_plane % 4)
    if currents_torch[ip] > 0:
        rect = plt.Rectangle((x_rect, y_rect),
                             sim.pixel_pitch, sim.pixel_pitch,
                             linewidth=0.1, fc=m_curr.to_rgba(currents_torch[ip]),
                             edgecolor='gray')
        ax.add_patch(rect)
        art3d.pathpatch_2d_to_3d(rect, z=sim.tpc_borders[pixel_plane][2][0], zdir="y")
        
anode1 = plt.Rectangle((sim.tpc_borders[0][0][0], sim.tpc_borders[0][1][0]),
                        sim.tpc_borders[0][0][1]-sim.tpc_borders[0][0][0], 
                        sim.tpc_borders[0][1][1]-sim.tpc_borders[0][1][0],
                       linewidth=1, fc='none',
                       edgecolor='gray', label=('Pixel' if ip == 5 else ''))
ax.add_patch(anode1)
art3d.pathpatch_2d_to_3d(anode1, z=sim.tpc_borders[0][2][0], zdir="y")

anode2 = plt.Rectangle((sim.tpc_borders[0][0][0], sim.tpc_borders[0][1][0]),
                        sim.tpc_borders[0][0][1]-sim.tpc_borders[0][0][0], 
                        sim.tpc_borders[0][1][1]-sim.tpc_borders[0][1][0],
                       linewidth=1, fc='none',
                       edgecolor='gray', label=('Pixel' if ip == 5 else ''))
ax.add_patch(anode2)
art3d.pathpatch_2d_to_3d(anode2, z=sim.tpc_borders[1][2][0], zdir="y")

cathode = plt.Rectangle((sim.tpc_borders[0][0][0], sim.tpc_borders[0][1][0]),
                        sim.tpc_borders[0][0][1]-sim.tpc_borders[0][0][0], 
                        sim.tpc_borders[0][1][1]-sim.tpc_borders[0][1][0],
                        linewidth=1, fc='gray', alpha=0.5,
                        edgecolor='gray', label=('Pixel' if ip == 5 else ''))
ax.add_patch(cathode)
art3d.pathpatch_2d_to_3d(cathode, z=0, zdir="y")

ax.plot((sim.tpc_borders[0][0][0],sim.tpc_borders[0][0][0]),(sim.tpc_borders[0][2][0],sim.tpc_borders[1][2][0]),
        (sim.tpc_borders[0][1][0],sim.tpc_borders[0][1][0]), lw=1,color='gray')

ax.plot((sim.tpc_borders[0][0][0],sim.tpc_borders[0][0][0]),(sim.tpc_borders[0][2][0],sim.tpc_borders[1][2][0]),
        (sim.tpc_borders[0][1][1],sim.tpc_borders[0][1][1]), lw=1,color='gray')

ax.plot((sim.tpc_borders[0][0][1],sim.tpc_borders[0][0][1]),(sim.tpc_borders[0][2][0],sim.tpc_borders[1][2][0]),
        (sim.tpc_borders[0][1][0],sim.tpc_borders[0][1][0]), lw=1,color='gray')

ax.plot((sim.tpc_borders[0][0][1],sim.tpc_borders[0][0][1]),(sim.tpc_borders[0][2][0],sim.tpc_borders[1][2][0]),
        (sim.tpc_borders[0][1][1],sim.tpc_borders[0][1][1]), lw=1,color='gray')

# ax.set_ylim(sim.module_borders[pixel_plane][2][0],50)
ax.set_xlim(sim.tpc_borders[0][0][0],sim.tpc_borders[1][0][1])
ax.set_ylim(sim.tpc_borders[0][2][0],sim.tpc_borders[1][2][0])
ax.set_zlim(sim.tpc_borders[0][1][0],sim.tpc_borders[1][1][1])

ax.set_box_aspect((2,2,4))
ax.grid(False)
ax.xaxis.set_major_locator(plt.MaxNLocator(3))
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
# ax.set_axis_off()
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

ax.view_init(azim=20)
# def rotate(angle):
#     ax.view_init(azim=angle)
    
# from matplotlib import animation

# rot_animation = animation.FuncAnimation(fig, rotate, frames=np.arange(0,362,2),interval=50)

ax.set_ylabel("z [cm]")
ax.set_xlabel("x [cm]")
ax.set_zlabel("y [cm]")
_ = plt.colorbar(m_curr,fraction=0.035, pad=0.05,label='Induced current integral [# electrons]')


In [ ]:
%matplotlib inline
plt.plot(np.arange(len(currents.get().flatten())), currents.get().flatten())
plt.plot(np.arange(len(currents_torch.flatten())), currents_torch.flatten())
plt.show()

Mostly quite close, cupy (blue) has an extra spike

In [ ]:
# rot_animation.save('animation.gif', writer='imagemagick', fps=10, bitrate=10, dpi=100)

### Electronics response and digitization 
Here we simulate the electronics response (the self-triggering cycle) and the signal digitization.

In [ ]:
time_ticks = cp.linspace(0, len(unique_eventIDs)*consts.time_interval[1]*3, pixels_signals.shape[1]+1)
integral_list = cp.zeros((pixels_signals.shape[0], fee.MAX_ADC_VALUES))
adc_ticks_list = cp.zeros((pixels_signals.shape[0], fee.MAX_ADC_VALUES))
TPB = 128
BPG = ceil(pixels_signals.shape[0] / TPB)
rng_states = create_xoroshiro128p_states(TPB * BPG, seed=0)
# integrate = cp.zeros((pixels_signals.shape[0], time_ticks.shape[0]))
fee.get_adc_values[BPG,TPB](pixels_signals,
                            time_ticks,
                            integral_list,
                            adc_ticks_list,
                            0,
                            rng_states)#,
adc_list = fee.digitize(integral_list)

In [ ]:
time_ticks_torch = torch.linspace(0, len(unique_eventIDs)*sim.time_interval[1]*3, pixels_signals_torch.shape[1]+1)

integral_list_torch, adc_ticks_list_torch = sim.get_adc_values(pixels_signals_torch,
                                                               time_ticks_torch,
                                                                0)
adc_list_torch = sim.digitize(integral_list_torch)

### 2D event display with induced current and ADC counts

In [ ]:
cmap = cm.Spectral_r
norm = colors.Normalize(vmin=0, vmax=256)
m = cm.ScalarMappable(norm=norm, cmap=cmap)

cmap = cm.viridis
norm_curr = colors.LogNorm(vmin=min(currents[currents>0]), vmax=max(currents))
m_curr = cm.ScalarMappable(norm=norm_curr, cmap=cmap)

fig,ax = plt.subplots(1,2,figsize=(6,3.8))

importlib.reload(detsim)
for ip, p in enumerate(unique_pix):
    x_rect, y_rect = detsim.get_pixel_coordinates(p.get())
    pixel_plane = int(p[0] // consts.n_pixels[0])
    c = currents[ip].get()
    if c >= 1:    
        rect = plt.Rectangle((x_rect, y_rect),
                             consts.pixel_pitch, consts.pixel_pitch,
                             linewidth=0.2, fc=m_curr.to_rgba(c),
                             edgecolor='grey')
        ax[0].add_patch(rect)

    a = adc_list[ip][adc_list[ip]>fee.digitize(0)]
    if len(a):    
        rect = plt.Rectangle((x_rect, y_rect),
                             consts.pixel_pitch, consts.pixel_pitch,
                             linewidth=0.2, fc=m.to_rgba(np.sum(a.get())),
                             edgecolor='grey')
        ax[1].add_patch(rect)


for it,t in enumerate(selected_tracks):
    ax[0].plot((t["x_start"], t["x_end"]), 
            (t["y_start"], t["y_end"]),
            c='r',
            lw=1.25,
            ls=':',
            alpha=1,
            zorder=10)
    ax[1].plot((t["x_start"], t["x_end"]), 
            (t["y_start"], t["y_end"]),
            c='r',
            lw=0,
            ls=':',
            alpha=1,
            zorder=10)
    ax[0].scatter((t["x_start"], t["x_end"]), 
                  (t["y_start"], t["y_end"]),
                  c='r', s=1, zorder=99999)
#     ax[1].scatter((t["x_start"], t["x_end"]), 
#                   (t["y_start"], t["y_end"]),
#                   c='r', s=1, zorder=99999)       

ax[0].set_aspect("equal")
ax[1].set_aspect("equal")
ax[0].set_xlabel("x [cm]")
ax[1].set_xlabel("x [cm]")
ax[0].set_ylabel("y [cm]")
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

divider0 = make_axes_locatable(ax[1])
cax0 = divider0.append_axes("right", size="7%", pad=0.07)
fig.colorbar(m, ax=ax[1], cax=cax0, label='ADC counts sum')

divider1 = make_axes_locatable(ax[0])
cax1 = divider1.append_axes("right", size="7%", pad=0.07)
fig.colorbar(m_curr, ax=ax[0], cax=cax1, label='Induced current integral [# electrons]')

plt.subplots_adjust(hspace=0.5)
fig.savefig("currentadc.pdf")

In [ ]:
cmap = cm.Spectral_r
norm = colors.Normalize(vmin=0, vmax=256)
m = cm.ScalarMappable(norm=norm, cmap=cmap)

#cmap = cm.viridis
#norm_curr = colors.LogNorm(vmin=min(currents_torch[currents_torch>0]), vmax=max(currents_torch))
#m_curr = cm.ScalarMappable(norm=norm_curr, cmap=cmap)

fig,ax = plt.subplots(1,2,figsize=(6,3.8))

x_r_all, y_r_all = sim.get_pixel_coordinates(ep.astensor(unique_pix_torch))
for ip, p in enumerate(unique_pix_torch):
    x_rect = x_r_all[ip, 0].numpy()
    y_rect = y_r_all[ip, 0].numpy()
    pixel_plane = int(p[0] // sim.n_pixels[0])
    c = currents_torch[ip]
    if c >= 1:    
        rect = plt.Rectangle((x_rect, y_rect),
                             sim.pixel_pitch, sim.pixel_pitch,
                             linewidth=0.2, fc=m_curr.to_rgba(c),
                             edgecolor='grey')
        ax[0].add_patch(rect)

    a = adc_list_torch[ip][adc_list_torch[ip]>sim.digitize(torch.tensor([0]))]
    if len(a):    
        rect = plt.Rectangle((x_rect, y_rect),
                             sim.pixel_pitch, sim.pixel_pitch,
                             linewidth=0.2, fc=m.to_rgba(a.sum()),
                             edgecolor='grey')
        ax[1].add_patch(rect)


for it,t in enumerate(selected_tracks):
    ax[0].plot((t["x_start"], t["x_end"]), 
            (t["y_start"], t["y_end"]),
            c='r',
            lw=1.25,
            ls=':',
            alpha=1,
            zorder=10)
    ax[1].plot((t["x_start"], t["x_end"]), 
            (t["y_start"], t["y_end"]),
            c='r',
            lw=0,
            ls=':',
            alpha=1,
            zorder=10)
    ax[0].scatter((t["x_start"], t["x_end"]), 
                  (t["y_start"], t["y_end"]),
                  c='r', s=1, zorder=99999)
#     ax[1].scatter((t["x_start"], t["x_end"]), 
#                   (t["y_start"], t["y_end"]),
#                   c='r', s=1, zorder=99999)       

ax[0].set_aspect("equal")
ax[1].set_aspect("equal")
ax[0].set_xlabel("x [cm]")
ax[1].set_xlabel("x [cm]")
ax[0].set_ylabel("y [cm]")
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

divider0 = make_axes_locatable(ax[1])
cax0 = divider0.append_axes("right", size="7%", pad=0.07)
fig.colorbar(m, ax=ax[1], cax=cax0, label='ADC counts sum')

divider1 = make_axes_locatable(ax[0])
cax1 = divider1.append_axes("right", size="7%", pad=0.07)
fig.colorbar(m_curr, ax=ax[0], cax=cax1, label='Induced current integral [# electrons]')

plt.subplots_adjust(hspace=0.5)
fig.savefig("currentadc_torch.pdf")

In [ ]:
cmap = cm.seismic
norm = colors.Normalize(vmin=0.5, vmax=1.5)
m = cm.ScalarMappable(norm=norm, cmap=cmap)

cmap = cm.seismic
norm_curr = colors.Normalize(vmin=0.5, vmax=1.5)
m_curr = cm.ScalarMappable(norm=norm_curr, cmap=cmap)

fig,ax = plt.subplots(1,2,figsize=(6,3.8))

x_r_all, y_r_all = sim.get_pixel_coordinates(ep.astensor(unique_pix_torch))
for ip, p in enumerate(unique_pix_torch):
    x_rect = x_r_all[ip, 0].numpy()
    y_rect = y_r_all[ip, 0].numpy()
    pixel_plane = int(p[0] // sim.n_pixels[0])
    c = currents_torch[ip].numpy()
    c_og = currents[ip].get()
    if c >= 1 or c_og >=1:    
        rect = plt.Rectangle((x_rect, y_rect),
                             sim.pixel_pitch, sim.pixel_pitch,
                             linewidth=0.2, fc=m_curr.to_rgba(c/c_og),
                             edgecolor='grey')
        ax[0].add_patch(rect)

    a = adc_list_torch[ip][adc_list_torch[ip]>sim.digitize(torch.tensor([0]))].numpy()
    a_og = adc_list[ip][adc_list[ip]>fee.digitize(0)].get()
    if len(a) or len(a_og):    
        rect = plt.Rectangle((x_rect, y_rect),
                             sim.pixel_pitch, sim.pixel_pitch,
                             linewidth=0.2, fc=m.to_rgba(a.sum()/a_og.sum()),
                             edgecolor='grey')
        ax[1].add_patch(rect)


for it,t in enumerate(selected_tracks):
    ax[0].plot((t["x_start"], t["x_end"]), 
            (t["y_start"], t["y_end"]),
            c='r',
            lw=1.25,
            ls=':',
            alpha=1,
            zorder=10)
    ax[1].plot((t["x_start"], t["x_end"]), 
            (t["y_start"], t["y_end"]),
            c='r',
            lw=0,
            ls=':',
            alpha=1,
            zorder=10)
    ax[0].scatter((t["x_start"], t["x_end"]), 
                  (t["y_start"], t["y_end"]),
                  c='r', s=1, zorder=99999)
#     ax[1].scatter((t["x_start"], t["x_end"]), 
#                   (t["y_start"], t["y_end"]),
#                   c='r', s=1, zorder=99999)       

ax[0].set_aspect("equal")
ax[1].set_aspect("equal")
ax[0].set_xlabel("x [cm]")
ax[1].set_xlabel("x [cm]")
ax[0].set_ylabel("y [cm]")
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

divider0 = make_axes_locatable(ax[1])
cax0 = divider0.append_axes("right", size="7%", pad=0.07)
fig.colorbar(m, ax=ax[1], cax=cax0, label='ADC counts sum Ratio (Diff / Original)')

divider1 = make_axes_locatable(ax[0])
cax1 = divider1.append_axes("right", size="7%", pad=0.07)
fig.colorbar(m_curr, ax=ax[0], cax=cax1, label='Induced current integral Ratio (Diff / Original)')

plt.subplots_adjust(hspace=0.5)
fig.savefig("currentadc_ratio.pdf")

Ratio is quite close, mod noise + one cluster of pixels (likely due to difference mentioned above)

### Export result
As a last step we backtrack the ADC counts to the Geant4 tracks and we export the result in a HDF5 file.

In [ ]:
MAX_TRACKS_PER_PIXEL = 5

# Mapping between unique pixel array and track array index
track_pixel_map = cp.full((unique_pix.shape[0], MAX_TRACKS_PER_PIXEL), -1)
TPB = 32
BPG = ceil(unique_pix.shape[0] / TPB)
detsim.get_track_pixel_map[BPG, TPB](track_pixel_map, unique_pix, neighboring_pixels)

# Here we backtrack the ADC counts to the Geant4 tracks
TPB = 128
BPG = ceil(adc_list.shape[0] / TPB)
backtracked_id = cp.full((adc_list.shape[0], adc_list.shape[1], MAX_TRACKS_PER_PIXEL), -1)

detsim.backtrack_adcs[BPG,TPB](selected_tracks,
                               adc_list,
                               adc_ticks_list,
                               track_pixel_map,
                               event_id_map,
                               unique_eventIDs,
                               backtracked_id,
                               0)

In [ ]:
!rm test.h5
pc = fee.export_to_hdf5(adc_list.get(),
                        adc_ticks_list.get(),
                        unique_pix.get(),
                        backtracked_id.get(),
                        "test.h5")

In [ ]:
!rm evd.test.h5
!python /global/homes/s/soleti/larpix/larpix-v2-testing-scripts/event-display/to_evd_file.py --in_filename test.h5  --out_filename evd.test.h5  --geometry_file ../larndsim/pixel_layouts/multi_tile_layout-2.2.16.yaml --event_dt 2000 --dbscan_eps 25 --vd 1.587 --nhit_cut 2

In [ ]:
evd = h5py.File("evd.test.h5")

In [ ]:
fig,ax=plt.subplots(1,1)
ax.plot((selected_tracks['z_start']*10,selected_tracks['z_end']*10),(selected_tracks['y_start']*10,selected_tracks['y_end']*10),c='k')
ax.plot((evd['tracks']['start'][:,2],evd['tracks']['end'][:,2]),(evd['tracks']['start'][:,1]-218.236,evd['tracks']['end'][:,1]-218.236),label='Reconstructed track',ls='--',c='r')
ax.legend()
ax.set_xlabel("z [mm]")
ax.set_ylabel("y [mm]")